# Spam detection Data Science Project
This project was inspired by the Chapter 3 exercises of the book 'Hands on Machine Learning' by Aurelien Geron, if you are new to Machile learning / Data Science, I highly recommend the book<br>
In order to make this project, I followed the line of thought present on the following link, by the same author cited above: https://github.com/ageron/handson-ml2/blob/master/03_classification.ipynb<br>
https://towardsdatascience.com/spam-detection-in-emails-de0398ea3b48
I avoided copying the above code, because doing so would hinder my learning, so if you want to check a better solution, you should check the link above<br>
The used dataset can be found in the following link: https://spamassassin.apache.org/old/publiccorpus/


## First, let's import the data

In [5]:
import os
import re

In [6]:
SPAM_PATH = './data/spam'
HAM_PATH = './data/easy_ham'

In [7]:
spam_filenames = os.listdir('./data/spam')
ham_filenames = os.listdir('./data/easy_ham')
# spam_filenames

In [8]:
import email
import email.policy

def load_email(filename, folder_path):
    with open(os.path.join(folder_path, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [9]:
ham_emails = [load_email(filename=name, folder_path=HAM_PATH) for name in ham_filenames]
spam_emails = [load_email(filename=name, folder_path=SPAM_PATH) for name in spam_filenames]

## Now it is time to clean the data. We need to remove hyperlinks, special characters and other stuff as well

In [10]:
print(ham_emails[0].get_content().strip())

Tim> About half the developers on the spambayes project were missing
    Tim> some permission or other, so I ran thru all of them and checked
    Tim> every damned box and clicked on every damn dropdown list I could
    Tim> find.  As far as SF is concerned, you're all sitting on God's Right
    Tim> Hand now, so if it still doesn't work I suggest you upgrade to
    Tim> Win98 <wink>.

Time to upgrade I guess. :-(

    % cvs add unheader.py 
    cvs [server aborted]: "add" requires write access to the repository

I'll try a checkout into a new directory...

S


In [11]:
ham_emails[1].get_payload()

"Hi All,\n\nAnyone ever try connecting at 1200bps in Linux? I've got a USR 56K\nFaxmodem which is meant to connect to another (same) modem and I have to\nconnect at this speed due to the (NT) port settings on the remote side,\nbut the modem handshake always fails at this speed.\n\nThe modem handshake works at slightly higher speeds (4800bps to\n~57600bps) but that is no good for tx/rx'ing data to the remote server\nas it insists at talking at the speed of treacle/1200bps.\n\nNote Minicom fails to handshake at 1200bps, but HyperTerm in Windows\nworked first time(!?), any ideas?\n\nBaud 1200 7 data bits Even Parity\n\nI am doing a ATZ3 to reset the modem then I send this init string:\n\nAT&F1E1V1Q0X4Y0S32=232&A1&B0&C1&D2&H0&I0&K1&M4&N0&P0&R1&S0&U0&Y1\n\n... which is most of the defaults.\n\nUSR said to set S15=128 (disables v.42)) \n& set S32=98 (disable v.92 & X2)\n\nBut the S15=128 just makes the handshake lockup instead of just giving\nup.\n\nbtw this is a bank's system I am connectin

In [12]:
import base64

In [13]:
def get_email_body(emailobj):
    """ Return the body of the email, preferably in text.
    """

    def _get_body(emailobj):
        """ Return the first text/plain body found if the email is multipart
        or just the regular payload otherwise.
        """
        if emailobj.is_multipart():
            for payload in emailobj.get_payload():
                # If the message comes with a signature it can be that this
                # payload itself has multiple parts, so just return the
                # first one
                if payload.is_multipart():
                    return _get_body(payload)

                body = payload.get_payload()
                if payload.get_content_type() == "text/plain":
                    return body
        else:
            return emailobj.get_payload()

    body = _get_body(emailobj)

    enc = emailobj["Content-Transfer-Encoding"]
    if enc == "base64":
        # body = base64.decodestring(body)
        body = "base64"

    return body 

In [14]:
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)

In [15]:
def clean_body(email):
    if(email):

        to_clean = ['\n', '@', '=', '!', '.', ',', '\'', ';', '(', ')', '<','>', ':','?', '"', '*', '1', '2', '3' ,  '4', '5', '6', '7', '8', '9','0', '+','^','~']
        to_space = ['-','_','/']
        result = re.sub(r"http\S+", "url", email)
        result = html_to_plain_text(result)
        # result = result.split()
        for sub in to_clean: result=result.replace(sub,"")
        for sub in to_space: result= result.replace(sub,' ')
        result = ' '.join(word for word in result.split(' ') if not word.startswith(('/', '%', 'r`')))
        # result = ''.join(digit for digit in result if not digit.isdigit())
        result = result.lower()
        return result
    
    else: return ""

In [16]:
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)

In [17]:
%%time
# clean_email = clean_body(get_email_body(ham_emails[3])).strip()
clean_email = clean_body(get_email_body(spam_emails[1])).strip()
clean_email
# get_email_body(ham_emails[3])

CPU times: user 1.52 ms, sys: 0 ns, total: 1.52 ms
Wall time: 1.54 ms


'take control of your computer with this top of the line software                 symantec systemworks                professional software suite this special package includes six   yes    feature packed utilitiesall for  special low price of only $this software will   protect your computer from unwanted and hazardous viruses   help secure your private & valuable information   allow you to transfer files and send e mails safely   backup your all your data quick and easily   improve your pcs performance w superior integral diagnostics    youll never have to take your pc to the repair shop again         thats six yes    feature packed utilities   great price    a $ combined retail value yours only $ limited time offer         why so cheap you ask you are buying online wholesale                  direct from the warehouse to you                               and          for a limited time buy  of any software & get  free            dont fall prey to destructive viruses or programs        

In [18]:
from collections import Counter
# count = Counter(clean_email.split())
# count

In [19]:
#para todo email vamos rodar o clean e o counter

In [20]:
emails = ham_emails + spam_emails
y = [0 for a in range(len(ham_emails))] + [1 for a in range(len(spam_emails))]

In [21]:
# len(ham_emails)
# emails[2550] == ham_emails[2550]
# Y[2551]

In [22]:
clean_spam = [get_email_body(email) for email in spam_emails]

In [23]:
html_to_plain_text(clean_spam[4])

"\n    Does Your Comp=\nuter Need an Oil\n      Change?\n    N=\nortonSystemWorks\n      2002 Professional\n      Edition\n    =\nMade\n      by the Creators of the #1 Anti-Virus Software on the Market!\n    This\n      UNBEATABLE software suite comes with  EVERY\n      program you'll  ever n=\need to answer the problems or threats that your\n      computer faces each day of it's Life!Included in this magnificent deal\n      are the following programs:\n    Norton\n      AntiVirus=FFFFFF99 2002 - THE #1\n      ANTI-VIRUS PROTECION EVER!Norton Utilities=FFFFFF99 2002\n      - DIAGNOSE ANY PROBLEM WI=\nTH YOUR\n      SYSTEM!\n      Norton Ghost=FFFFFF99 2002 - MAKES\n      BACKING UP YOUR VALUABLE DATA EASY!\n      Norton CleanSweep=FFFFFF99 2002 - CLEANS\n      OUT EXCESS INTERNET FILE BUILDUP!\n      Norton WinFax=FFFFFF99 Basic - TURNS YOUR\n      CPU INTO A FAX MACHINE!\n      GoBack=FFFFFFAE 3 Personal - HELPS\n      PREVENT YOU FROM MAKING ANY MISTAKES!\n    *ALL\n      this has a 

In [24]:
clean_spam[4]

'<html>\n\n<head>\n<meta http-equiv=3D"Content-Language" content=3D"en-us">\n<meta http-equiv=3D"Content-Type" content=3D"text/html; charset=3Dwindows-=\n1252">\n<meta name=3D"GENERATOR" content=3D"Microsoft FrontPage 4.0">\n<meta name=3D"ProgId" content=3D"FrontPage.Editor.Document">\n<title>Does Your Computer Need an Oil Change</title>\n</head>\n\n<body>\n\n<table border=3D"0" width=3D"538" height=3D"1">\n  <tr>\n    <td width=3D"538" height=3D"1" align=3D"center" bgcolor=3D"#000000"><b=\n><font face=3D"Century Gothic" size=3D"5" color=3D"#FFFFFF">Does Your Comp=\nuter Need an Oil\n      Change?</font></b></td>\n  </tr>\n</table>\n<table border=3D"0" width=3D"538" height=3D"151">\n  <tr>\n    <td width=3D"530" height=3D"145"><b><font face=3D"Tahoma" size=3D"5">N=\norton</font><font color=3D"#006600" face=3D"Verdana" size=3D"7"><br></font=\n><i><font face=3D"Verdana" color=3D"#CC0000" size=3D"7">SystemWorks\n      2002</font></i><font size=3D"4" face=3D"Verdana"><br> </font><font f=\n

In [25]:
# counter_list = [Counter(clean_body(get_email_body(email)).strip().split()) for email in ham_emails]

In [26]:
counter_list =  [Counter(clean_body(get_email_body(email)).strip().split()) for email in emails]

In [27]:
## convert HTML to plain text -> beautifulsoup

In [28]:
import pandas as pd
# df = pd.DataFrame(counter_list)
# df.head()

In [29]:
# df.fillna(value=0,inplace=True)

### Huge amount of features, there is still room for a lot of cleaning
### What if we grouped together the words that have kind of the same meaning, like play and playing?

In [30]:
import nltk
stemmer = nltk.PorterStemmer()


def stem_counter(counter_var):
    stemmed_word_counts = Counter()
    for word, count in counter_var.items():
        stemmed_word = stemmer.stem(word)
        stemmed_word_counts[stemmed_word] += count
        # print(len(stemmed_word_counts))
    return stemmed_word_counts

# eu quero uma lista dos novos counters
# new_counter_list = [stem_counter(counter) for counter in counter_list[:]]
   

In [31]:
new_counter_list = [stem_counter(counter) for counter in counter_list]

In [32]:
# new_counter_list[0]


In [33]:
import pandas as pd

In [34]:
df = pd.DataFrame(new_counter_list)
df.head()

,tim,about,half,the,develop,on,spambay,project,were,miss,...,emiratesunit,kingdomuruguayuzbekistanvanuatuvatican,cityvenezuelavietnamvirgin,islandsw,futuna,islandswestern,saharayemenyugoslavia,formerzairezambiazimbabwetelephonemobil,workfaxemail,jchoatenewyorkcom
0,6.0,1.0,1.0,3.0,1.0,3.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,9.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,NaN,11.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,17.0,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Well, that was a nice improvement, from 58426 to 50172, features, we reduced it by 15% percent.

In [35]:
df.fillna(value=0,inplace=True)

In [36]:
from sklearn.model_selection import train_test_split
X = df
# y = y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
# log_clf = LogisticRegression(solver="lbfgs", max_iter=10, random_state=42)
# log_clf.fit(X_train,y_train)
log_clf = LogisticRegression(solver="lbfgs", max_iter=100, random_state=42)
score = cross_val_score(log_clf, X_train, y_train, cv=3, verbose=3)
score.mean()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
